In [1]:
#Importing all the neccessary libraries required 
import os
import pandas as pd
import codecs
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
import numpy as np  # Numpy library for creating and modifying arrays.
from keras.layers import Dense, SimpleRNN, GRU, LSTM, Embedding # Import layers from Keras
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import nltk

Using TensorFlow backend.


In [2]:
#fetching action files
path_to_Action = '/home/sandy5290/Downloads/Dataset/TrainDataset/Action/'
Action_files = [Action for Action in os.listdir(path_to_Action) if Action.endswith('.txt')]

#fetching comedy files
path_to_Comedy = '/home/sandy5290/Downloads/Dataset/TrainDataset/Comedy/'
Comedy_files = [Comedy for Comedy in os.listdir(path_to_Comedy) if Comedy.endswith('.txt')]

#fetching Crime files
path_to_Crime = '/home/sandy5290/Downloads/Dataset/TrainDataset/Crime/'
Crime_files = [Crime for Crime in os.listdir(path_to_Crime) if Crime.endswith('.txt')]

#fetching Horror files
path_to_Horror = '/home/sandy5290/Downloads/Dataset/TrainDataset/Horror/'
Horror_files = [Horror for Horror in os.listdir(path_to_Horror) if Horror.endswith('.txt')]

#fetching Musical files
path_to_Musical = '/home/sandy5290/Downloads/Dataset/TrainDataset/Musical/'
Musical_files = [Musical for Musical in os.listdir(path_to_Musical) if Musical.endswith('.txt')]

#fetching Romance files
path_to_Romance = '/home/sandy5290/Downloads/Dataset/TrainDataset/Romance/'
Romance_files = [Romance for Romance in os.listdir(path_to_Romance) if Romance.endswith('.txt')]

#fetching War files
path_to_War = '/home/sandy5290/Downloads/Dataset/TrainDataset/War/'
War_files = [War for War in os.listdir(path_to_War) if War.endswith('.txt')]

In [3]:
#creating dataframe to store action subtitles
action_df = pd.DataFrame(columns=['Subtitle','Genre'])

#reading each file and removing nextline tags,punctuations from the file which are not required
for index,txt in enumerate(Action_files):
    act_file = codecs.open(os.path.join(path_to_Action,txt),encoding = "ISO-8859-1")
    Sub = act_file.read().replace("\n"," ")
    Subtitle = Sub.translate(str.maketrans('','',string.punctuation))
    Genre = 'Action'
    action_df.loc[index] = [Subtitle,Genre]

In [4]:
#dropping 3rd index since it contained nly special characters
action_df.drop(index=3,inplace=True)

In [5]:
#resetting the index
action_df.reset_index(drop=True,inplace=True)

In [6]:
#Repeating same for different genre
comedy_df = pd.DataFrame(columns=['Subtitle','Genre'])

for index,txt in enumerate(Comedy_files):
    com_file = codecs.open(os.path.join(path_to_Comedy,txt),encoding = "ISO-8859-1")
    Sub = com_file.read().replace("\n"," ")
    Subtitle = Sub.translate(str.maketrans('','',string.punctuation))
    Genre = 'Comedy'
    comedy_df.loc[index] = [Subtitle,Genre]

In [7]:
crime_df = pd.DataFrame(columns=['Subtitle','Genre'])

for index,txt in enumerate(Crime_files):
    cri_file = codecs.open(os.path.join(path_to_Crime,txt),encoding = "ISO-8859-1")
    Sub = cri_file.read().replace("\n"," ")
    Subtitle = Sub.translate(str.maketrans('','',string.punctuation))
    Genre = 'Action'
    crime_df.loc[index] = [Subtitle,Genre]

In [8]:
horror_df = pd.DataFrame(columns=['Subtitle','Genre'])

for index,txt in enumerate(Horror_files):
    hor_file = codecs.open(os.path.join(path_to_Horror,txt),encoding = "ISO-8859-1")
    Sub = hor_file.read().replace("\n"," ")
    Subtitle = Sub.translate(str.maketrans('','',string.punctuation))
    Genre = 'Horror'
    horror_df.loc[index] = [Subtitle,Genre]

In [9]:
musical_df = pd.DataFrame(columns=['Subtitle','Genre'])

for index,txt in enumerate(Musical_files):
    mus_file = codecs.open(os.path.join(path_to_Musical,txt),encoding = "ISO-8859-1")
    Sub = mus_file.read().replace("\n"," ")
    Subtitle = Sub.translate(str.maketrans('','',string.punctuation))
    Genre = 'Musical'
    musical_df.loc[index] = [Subtitle,Genre]

In [10]:
romance_df = pd.DataFrame(columns=['Subtitle','Genre'])

for index,txt in enumerate(Romance_files):
    rom_file = codecs.open(os.path.join(path_to_Romance,txt),encoding = "ISO-8859-1")
    Sub = rom_file.read().replace("\n"," ")
    Subtitle = Sub.translate(str.maketrans('','',string.punctuation))
    Genre = 'Romance'
    romance_df.loc[index] = [Subtitle,Genre]

In [11]:
war_df = pd.DataFrame(columns=['Subtitle','Genre'])

for index,txt in enumerate(War_files):
    war_file = codecs.open(os.path.join(path_to_War,txt),encoding = "ISO-8859-1")
    Sub = war_file.read().replace("\n"," ")
    Subtitle = Sub.translate(str.maketrans('','',string.punctuation))
    Genre = 'War'
    war_df.loc[index] = [Subtitle,Genre]

In [12]:
war_df.head()

,Subtitle,Genre
0,Muvuca Zohar Group This is the story of a squ...,War
1,Cambodia To many Westerners it seemed a paradi...,War
2,Range me Three to five on the legs Theyre comi...,War
3,MRS TEASDALE I ASK YOU TO RECONSIDER YES YOUR ...,War
4,the allied landing in Europe had begun under c...,War


In [13]:
#Stopwords are those words which do not give any useful information about the content. 
#So we need to remove stopwords. We fetch stopwords from built in function
stopwords = [set(stopwords.words('english'))]

In [14]:
#appending all the dataframes in single dataframe which will include all genre
final_data = action_df.append([comedy_df,crime_df,horror_df,musical_df,romance_df,war_df])

In [15]:
#Reseting the index
final_data.reset_index(drop=True,inplace=True)

In [16]:
#Displaying first 5 rows
final_data.head()

,Subtitle,Genre
0,Joe youre under arrest Maybe so but dont turn ...,Action
1,SUNNI Whats going on Are you getting this FITZ...,Action
2,THE GREY A job at the end of the world A salar...,Action
3,Marcus My God Brother what have you done Yet ...,Action
4,Visit birdhdinfo for more m720p Movies Encoded...,Action


In [17]:
def tokenize(sentence):
    """Tokenizes given string.
    :param sentence: string to tokenize
    :return: list of string with tokens
    """
    sentence_tokenized  = nltk.tokenize.word_tokenize(sentence.replace("\n", " "))
    return [token.lower() for token in sentence_tokenized]

In [18]:
def remove_puncs_numbers_stop_words(tokens):
    """Remove punctuations in the words, words including numbers and words in the stop_words list.
    :param tokens: list of string
    :return: list of string with cleaned version
    """
    tokens          = [token.replace("'", '') for token in tokens]
    tokens_cleaned  = [token for token in tokens if token.isalpha() and token not in stopwords]
    return tokens_cleaned

In [19]:
def stem(tokens):
    """Stems the tokens with nltk SnowballStemmer
    :param tokens: list of string
    :return: list of string with words stems
    """
    stemmer         = nltk.SnowballStemmer(language='english')
    tokens_stemmed  = [stemmer.stem(token) for token in tokens]
    return tokens_stemmed

In [20]:
def preprocess_document(document):
    """Calls methods _tokenize, _remove_puncs_numbers_stop_words and _stem respectively.
    :param document: string to preprocess
    :return: string with processed version
    """
    doc_tokenized   = tokenize(document)
    doc_cleaned     = remove_puncs_numbers_stop_words(doc_tokenized)
    doc_stemmed     = stem(doc_cleaned)
    doc_stemmed_str = ' '.join(doc_stemmed)
    return doc_stemmed_str

In [21]:
final_data['Subtitle'] = final_data['Subtitle'].apply(lambda x:preprocess_document(x))

In [22]:
final_data.head()

,Subtitle,Genre
0,joe your under arrest mayb so but dont turn ar...,Action
1,sunni what go on are you get this fitzi whoa w...,Action
2,the grey a job at the end of the world a salar...,Action
3,marcus my god brother what have you done yet a...,Action
4,visit birdhdinfo for more movi encod by bird t...,Action


In [23]:
#removing the tags
final_data['Subtitle'] = final_data['Subtitle'].str.replace('nbsp|âª|ltigt|apos|shehe|yã|não|yoï','')

In [24]:
#fetching all the words from data
tfidf = final_data['Subtitle'].tolist()

In [118]:
#removing words that occur in more than 80% documents
cv=CountVectorizer(max_df=0.8)
word_count_vector=cv.fit_transform(tfidf)

In [84]:
word_count_vector

<4828x174298 sparse matrix of type '<class 'numpy.int64'>'
	with 4135435 stored elements in Compressed Sparse Row format>

In [85]:
#calculating idf for all the words/vocab
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [86]:
#storing all the vocabs in variable
feature_names=cv.get_feature_names()

In [87]:
#defining the functions which will sort words based on tf-idf and fetch top 50 words from each document

def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items,topn=50):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [88]:
#function which fetches keywords along with tf-idf value
def get_keywords(idx):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([tfidf[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n;
    keywords=extract_topn_from_vector(feature_names,sorted_items)
    
    return keywords

In [89]:
#keeping only those words which had high tf-idf value
cleaned_text = pd.DataFrame(columns = ['Cleaned_Subs'])
for idx in range(0,final_data.shape[0]):
    key_wrds = get_keywords(idx)
    Pure = list(key_wrds.keys())
    cleaned_text.loc[idx] = [Pure]

In [90]:
#converting list to string
cleaned_text['Cleaned_Subs'] = cleaned_text['Cleaned_Subs'].apply(lambda x : ' '.join(x))

In [91]:
#merging the dataframes
cleaned_data = final_data.join(cleaned_text)

In [92]:
#dropping column which was not required
cleaned_data.drop(columns = ['Subtitle'],inplace=True)

In [93]:
#final dataframe for model building
cleaned_data.head()

,Genre,Cleaned_Subs
0,Action,burdett sheriff stumpi wheeler carlo consuelo ...
1,Action,fitzi holden nora zahir sunni pyramid shadid o...
2,Action,fuck ottway henrick talget flanneri diaz burk ...
3,Action,viktor selen marcus kraven lycan william tani ...
4,Action,loki tesseract stark furi romanoff coulson sel...


In [94]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

In [ ]:
#lemmatizing the words to get the root form of word
#cleaned_data['Cleaned_Subs'] = cleaned_data['Cleaned_Subs'].apply(lambda x:' '.join(lemmatize_text(x)))

In [95]:
#resetting the index
cleaned_data = cleaned_data.sample(frac=1).reset_index(drop=True)
cleaned_data.head()

,Genre,Cleaned_Subs
0,Comedy,améli poulain bretodeau bredoteau dominiqu suz...
1,Horror,leona harrigan aah danni fuck mike willi heine...
2,Romance,hodor euron bran sansa stark ramsay meera grey...
3,Horror,krissi jane evey coupland brian harri joseph d...
4,Musical,hojo sidhu wiii lushang kung chowk chandni fu ...


In [96]:
max_num_words = 50000
seq_len = 100
embedding_size = 200

In [97]:
#spliting data into train and test/val
X_train, X_test, y_train, y_test = train_test_split(cleaned_data['Cleaned_Subs'],cleaned_data['Genre'],stratify = cleaned_data['Genre'] ,test_size=0.3)

In [98]:
#
tokenizer = Tokenizer(num_words=max_num_words) #Tokenizer is used to tokenize text
tokenizer.fit_on_texts(X_train) #Fit this to our corpus

x_train = tokenizer.texts_to_sequences(X_train) #'text to sequences converts the text to a list of indices
x_train = pad_sequences(x_train, maxlen=100) #pad_sequences makes every sequence a fixed size list by padding with 0s 

x_test = tokenizer.texts_to_sequences(X_train) 
x_test = pad_sequences(x_test, maxlen=100)

x_train.shape, x_test.shape

((3379, 100), (3379, 100))

In [99]:
unique_labels = list(y_train.unique())
print(unique_labels)

['Musical', 'War', 'Romance', 'Action', 'Horror', 'Comedy']


In [100]:
u_lab_tst = list(y_test.unique())
print(u_lab_tst)

['Action', 'War', 'Horror', 'Romance', 'Musical', 'Comedy']


In [101]:
 # This convers the labels to one-hot vectors(Dummies)

y_train_n = np.array([unique_labels.index(i) for i in y_train]) # Convert the word labels to indeces
y_train_n = to_categorical(y_train_n) # Dummify the labels
y_test_n = np.array([unique_labels.index(i) for i in y_test])
y_test_n = to_categorical(y_test_n)

In [102]:
# Building an LSTM model
model = Sequential() # Call Sequential to initialize a network
model.add(Embedding(input_dim = max_num_words, 
                    input_length = seq_len, 
                    output_dim = embedding_size)) # Add an embedding layer which represents each unique token as a vector
model.add(LSTM(10,return_sequences=True)) # Add an LSTM layer
model.add(LSTM(10,return_sequences=False))
#model.add(LSTM(10, return_sequences=False))
model.add(Dense(6, activation='softmax')) # Add an ouput layer. .

In [103]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 200)          10000000  
_________________________________________________________________
lstm_5 (LSTM)                (None, 100, 10)           8440      
_________________________________________________________________
lstm_6 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 66        
Total params: 10,009,346
Trainable params: 10,009,346
Non-trainable params: 0
_________________________________________________________________


In [104]:
#setting learning rate value
adam = Adam(lr=0.001)

In [105]:
#callbacks are useful to monitor val_loss. If val_loss doesn't improve for continuous 5 epochs then it will stop executing.
#Indicating that there is no need to run more epochs as the val_loss is not improving
callbacks = [EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

In [106]:
# Mention the optimizer, Loss function and metrics to be computed
model.compile(optimizer=adam,                  # 'Adam' is a variant of gradient descent technique
              loss='categorical_crossentropy', # categorical_crossentropy for multi-class classification
              metrics=['accuracy'])            # These metrics are computed for evaluating and stored in history

model.fit(x_train, y_train_n, epochs=100,callbacks = callbacks ,validation_split=0.25,verbose=1)

Train on 2534 samples, validate on 845 samples
Epoch 1/100
2534/2534 [==============================] - 20s 8ms/step - loss: 1.7693 - acc: 0.2723 - val_loss: 1.7168 - val_acc: 0.2947
Epoch 2/100
2534/2534 [==============================] - 19s 8ms/step - loss: 1.3672 - acc: 0.5320 - val_loss: 1.5223 - val_acc: 0.4331
Epoch 3/100
2534/2534 [==============================] - 18s 7ms/step - loss: 0.8605 - acc: 0.8129 - val_loss: 1.6351 - val_acc: 0.3621
Epoch 4/100
2534/2534 [==============================] - 18s 7ms/step - loss: 0.5609 - acc: 0.9175 - val_loss: 1.7355 - val_acc: 0.3870
Epoch 5/100
2534/2534 [==============================] - 16s 6ms/step - loss: 0.3981 - acc: 0.9428 - val_loss: 1.9113 - val_acc: 0.3870
Epoch 6/100
2534/2534 [==============================] - 16s 7ms/step - loss: 0.3257 - acc: 0.9479 - val_loss: 2.0079 - val_acc: 0.3929
Epoch 7/100
2534/2534 [==============================] - 19s 7ms/step - loss: 0.2784 - acc: 0.9487 - val_loss: 2.0795 - val_acc: 0.3917


In [107]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(cleaned_data.Cleaned_Subs)
y = cleaned_data.Genre
print(y.shape)
print(x.shape)

(4828,)
(4828, 52056)


In [108]:
#converting labels to numeric 
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

In [109]:
from keras.utils import to_categorical
y_cat = to_categorical(y)

In [110]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y_cat, test_size=0.2, random_state=2)

In [111]:
num_classes = 6
input_shape = X_train.shape[1]
nb_epochs = 100

In [112]:
y_train[0]

array([0., 0., 0., 1., 0., 0.], dtype=float32)

In [113]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [114]:
model = Sequential() # This initializes a sequential model to which we can keep adding layers.
model.add(Dense(75, kernel_initializer='uniform', input_dim = input_shape, activation='relu'))
#model.add(Dense(70, kernel_initializer='uniform', input_dim = input_shape, activation='relu'))
#model.add(Dense(256, kernel_initializer='uniform', input_dim = input_shape, activation='relu'))# Add a dense layer 
model.add(Dense(6, kernel_initializer='uniform', activation='softmax'))

In [115]:
callbacks = [EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

In [116]:
# Setting learning and momentum
# Adam is the optimizer which is the state of the art Gradient Descent variation. 
from keras.optimizers import Adam
adam = Adam(lr=0.001)

model.compile(loss='categorical_crossentropy', # CrossEntropy is the loss function. 
              optimizer=adam,                  # Mention the optimizer
              metrics=['accuracy']) 

In [117]:
history = model.fit(X_train, y_train, epochs=nb_epochs,callbacks = callbacks, batch_size=32, validation_split=0.1) 

Train on 3475 samples, validate on 387 samples
Epoch 1/100
3475/3475 [==============================] - 8s 2ms/step - loss: 1.7371 - acc: 0.2817 - val_loss: 1.6868 - val_acc: 0.2455
Epoch 2/100
3475/3475 [==============================] - 7s 2ms/step - loss: 1.3485 - acc: 0.4483 - val_loss: 1.3908 - val_acc: 0.4806
Epoch 3/100
3475/3475 [==============================] - 7s 2ms/step - loss: 0.7320 - acc: 0.8725 - val_loss: 1.2798 - val_acc: 0.5323
Epoch 4/100
3475/3475 [==============================] - 7s 2ms/step - loss: 0.3558 - acc: 0.9249 - val_loss: 1.2964 - val_acc: 0.5452
Epoch 5/100
3475/3475 [==============================] - 7s 2ms/step - loss: 0.2100 - acc: 0.9321 - val_loss: 1.3528 - val_acc: 0.5530
Epoch 6/100
3475/3475 [==============================] - 7s 2ms/step - loss: 0.1579 - acc: 0.9304 - val_loss: 1.4277 - val_acc: 0.5349
Epoch 7/100
3475/3475 [==============================] - 7s 2ms/step - loss: 0.1366 - acc: 0.9329 - val_loss: 1.4742 - val_acc: 0.5271
Epoch 8/